# Implementación de los algoritmos genéticos


## Genes

Los genes representan el menor pedazo que compone una solución. Cada característica de una solución debe tener un conjunto de valores posibles. Entonces, los genes deben tener una forma de cambiar su valor, esto se hará mediante el método `mutate` que devolverá una nueva instancia del Gen con un nuevo valor.

In [289]:
from uuid import uuid4
from typing import *
from abc import ABC, abstractmethod


class Gene(ABC):
    """This class represents the smallest unit of a part of a problem"""

    def __init__(
        self,
        value,
        identifier: str = str(uuid4()),
    ) -> None:
        self.value = value
        """This is the value of the gene"""
        self.identifier: str = identifier
        """The unique identifier of a gene"""

    @abstractmethod
    def mutate(self) -> Self:
        """This method returns a new gene with a new value"""
        pass

    @abstractmethod
    def clone(self) -> Self:
        """This method returns an identical copy of this gene"""
        pass

    def __eq__(self, value: object) -> bool:
        return isinstance(value, Gene) and self.identifier == value.identifier

    def __hash__(self) -> int:
        return hash(self.identifier)

    def __str__(self) -> str:
        return str(self.value)

Ahora implementemos un gen muy sencillo que solamente tendrá una lista de opciones para escoger y su valor será una de estas opciones.

In [290]:
from uuid import uuid4
import random as rnd


class ChoiceGene(Gene):
    """This gene selects a value from a pool of available values"""

    def __init__(
        self, available_values: list | str, value=None, identifier: str = str(uuid4())
    ) -> None:
        if available_values is None:
            raise Exception(f"The available values is None")
        if len(available_values) == 0:
            raise Exception(
                "The list of available values must have at least one element"
            )
        if value is None:
            value = rnd.choice(available_values)
        super().__init__(value, identifier)
        self.available_values = available_values
        """The list of available values for choose"""

    def mutate(self) -> Self:
        v1, v2 = rnd.sample(self.available_values, 2)
        new_value = v1 if self.value == v2 else v2
        return ChoiceGene(self.available_values, new_value, self.identifier)

    def clone(self) -> Self:
        return ChoiceGene(self.available_values, self.value, self.identifier)

Probemos este nuevo gen implementado

In [291]:
available_letters = " abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ=?!-*+."

letter_gene = ChoiceGene(available_values=available_letters)
print(letter_gene)

y


## Cromosoma

Un cromosoma representa una solución a un problema y este está formado por pedazos de soluciones aka `Genes`. Un cromosoma debe tener alguna funcionalidad para decodificar sus genes y devolver una solución en términos del problema original.


In [292]:
class Chromosome(ABC):
    """This is the base class for chromosomes"""

    def __init__(
        self,
        genes: list[Gene],
        maximization_problem: bool,
    ) -> None:
        self.genes: list[Gene] = genes
        """This are the genes of the chromosome"""
        self.maximization_problem: bool = maximization_problem
        """Says if this chromosome is part of a maximization problem"""

    @abstractmethod
    def mate(self, other: Self) -> Self:
        """This method mates a chromosome with another chromosome"""
        pass

    @abstractmethod
    def mutate(self) -> Self:
        """This method returns a copy of this chromosome but with mutated genes"""
        pass

    @abstractmethod
    def improve(self, n_iterations: int = 10) -> Self:
        """This method improve a chromosome"""
        pass

    def get_best(self, other: Self) -> Self:
        """This method returns the best of the 2 chromosomes"""
        my_fitness = self.get_fitness()
        other_fitness = other.get_fitness()

        def cmp(x: float, y: float):
            return x > y if self.maximization_problem else x < y

        better = self if cmp(my_fitness, other_fitness) else other
        return better

    def get_solution_values(self) -> list:
        """This method returns a list with the values of all the genes in the chromosome"""
        return [gene.value for gene in self.genes]

    @abstractmethod
    def get_fitness(self) -> float:
        """This method returns the fitness of this chromosome"""
        pass

    @abstractmethod
    def compatible(self, other: Self) -> bool:
        """This method says if the other Chromosome is compatible for mating"""
        pass

    @abstractmethod
    def random_instance(self) -> Self:
        """This method returns a random instance of the chromosome"""
        pass

    @abstractmethod
    def get_solution(self) -> Any:
        """This method construct a solution in the terms of the problem.
        If you want to use this function then you should create a chromosome that implements it
        """
        pass

Ahora creemos un cromosoma genérico para codificar de forma sencilla las soluciones de algunos problemas.

In [293]:
from typing import *


class SimpleChromosome(Chromosome):
    """This chromosome performs random single point crossover and random simple mutation"""

    def __init__(
        self,
        genes: list[Gene],
        fitness_function: Callable[[list], float],
        maximization_problem: bool,
    ) -> None:
        super().__init__(genes, maximization_problem)
        self.fitness_function: Callable[[list], float] = fitness_function
        """The function used to calculate the fitness of a chromosome"""
        self.fitness_value: float = None
        """The fitness of this chromosome"""

    def random_instance(self) -> Self:
        new_genes = [gene.mutate() for gene in self.genes]
        return SimpleChromosome(
            new_genes, self.fitness_function, self.maximization_problem
        )

    def compatible(self, other: Self) -> bool:
        return len(self.genes) == len(other.genes) and all(
            type(gene1) == type(gene2) and gene1 == gene2
            for gene1, gene2 in zip(self.genes, other.genes)
        )

    def get_fitness(self) -> float:
        if self.fitness_value is not None:
            return self.fitness_value
        solution_values = self.get_solution_values()
        self.fitness_value = self.fitness_function(solution_values)
        return self.fitness_value

    def improve(self, n_iterations: int = 10) -> Self:
        best_chromosome = self
        for _ in range(n_iterations):
            new_chromosome = self.mutate()
            best_chromosome = best_chromosome.get_best(new_chromosome)
        return best_chromosome

    def mutate(self) -> Self:
        index = rnd.randrange(0, len(self.genes))
        old_gene = self.genes[index]
        mutated = old_gene.mutate()
        cloned_genes = [gene.clone() for gene in self.genes]
        cloned_genes[index] = mutated
        return SimpleChromosome(
            cloned_genes, self.fitness_function, self.maximization_problem
        )

    def get_solution(self) -> Any:
        return self.get_solution_values()

    def cloned_genes(self) -> list[Gene]:
        """This method returns a copy of all the genes of this chromosome"""
        return [gene.clone() for gene in self.genes]

    def mate(self, other: Self) -> Self:
        if not self.compatible(other):
            raise Exception(
                "The chromosomes are not compatible. Check the length of it's genes and the type of each one"
            )
        pivot_point = rnd.randrange(0, len(self.genes))
        my_genes, other_genes = (self.cloned_genes(), other.cloned_genes())
        first1, last1 = (my_genes[:pivot_point], my_genes[pivot_point:])
        first2, last2 = (other_genes[:pivot_point], other_genes[pivot_point:])
        child1 = SimpleChromosome(
            first1 + last2, self.fitness_function, self.maximization_problem
        )
        child2 = SimpleChromosome(
            first2 + last1, self.fitness_function, self.maximization_problem
        )
        return child1.get_best(child2)

## Algoritmo Genético

Ahora hay que implementar la lógica de los algoritmos genéticos. Toda implementación de un algoritmo genético debe dar una forma de dada una población elegir los padres de la futura generación.

In [294]:
class GeneticAlgorithm(ABC):
    """Base class for all implementations of Genetic Algorithms"""

    def __init__(self, sample_chromosome: Chromosome) -> None:
        self.sample_chromosome: Chromosome = sample_chromosome
        """The chromosome that represents a solution in the genetic algorithm"""
        self.maximization_problem: bool = self.sample_chromosome.maximization_problem
        """Tells if the problem to solve is a maximization problem"""

    @abstractmethod
    def selection(
        self,
        population: list[Chromosome],
    ) -> list[tuple[Chromosome, Chromosome]]:
        """This method returns a list of tuples where each tuple represents the parents to mate.
        Remember each chromosome has the fitness function inside"""
        pass

    def mate_population(self, population: list[Chromosome]) -> list[Chromosome]:
        """This method generate a new population of individuals from a population.
        This method calls internally the selection and the cross_over method"""
        new_population: list[Chromosome] = []
        parents: list[tuple[Chromosome, Chromosome]] = self.selection(population)
        for parent1, parent2 in parents:
            child: Chromosome = parent1.mate(parent2)
            new_population.append(child)
        return new_population

    @abstractmethod
    def solve(
        self,
        population_size: int,
        max_iterations: int,
        stop_criteria: Callable[[float], bool],
    ) -> Chromosome:
        """This method returns the best chromosome for the problem"""
        pass

In [295]:
from typing import Callable


class TournamentGeneticAlgorithm(GeneticAlgorithm):
    def __init__(
        self,
        sample_chromosome: Chromosome,
        tournament_percentage: float,
    ) -> None:
        super().__init__(sample_chromosome)
        if not 0 < tournament_percentage <= 1:
            raise Exception(
                "The tournament percentage must be between 0 (exclusive) and 1 (inclusive)"
            )
        self.tournament_percentage = tournament_percentage
        """The percentage of the population to take as participants in a tournament"""

    def generate_population(self, population_size: int) -> list[Chromosome]:
        """This method generates a new population"""
        population = [
            self.sample_chromosome.random_instance() for _ in range(population_size)
        ]
        return population

    def get_best(
        self,
        population_sample: list[Chromosome],
    ) -> Chromosome:
        best = population_sample[0]
        for i in range(1, len(population_sample)):
            best = best.get_best(population_sample[i])
        return best

    def select_next_parent_with_tournament(
        self,
        population: list[Chromosome],
        tournament_size: int,
    ) -> Chromosome:
        """This method returns a parent by the tournament selection method"""
        if tournament_size > len(population):
            raise Exception("The tournament size is greater than the population")
        tournament_population = rnd.sample(population, tournament_size)
        best = self.get_best(tournament_population)
        return best

    def selection(
        self, population: list[Chromosome]
    ) -> list[tuple[Chromosome, Chromosome]]:
        tournament_size = max(1, int(len(population) * self.tournament_percentage))
        parents: list[tuple[Chromosome, Chromosome]] = []
        for _ in range(len(population)):
            parent1 = self.select_next_parent_with_tournament(
                population, tournament_size
            )
            parent2 = self.select_next_parent_with_tournament(
                population, tournament_size
            )
            parents.append((parent1, parent2))
        return parents

    def improve_population(self, population: list[Chromosome]) -> list[Chromosome]:
        """Improves a population"""
        improved_chromosomes: list[Chromosome] = []
        for chromosome in population:
            improved = chromosome.improve()
            improved_chromosomes.append(improved)
        return improved_chromosomes

    def solve(
        self,
        population_size: int,
        max_iterations: int,
        stop_criteria: Callable[[float], bool],
    ) -> Chromosome:
        if population_size <= 5:
            raise Exception(
                "The population size must be greater than 5 for the algorithm to work"
            )
        if max_iterations <= 5:
            raise Exception("The maximum number of iterations must be greater than 5")
        population = self.improve_population(self.generate_population(population_size))
        best = self.get_best(population)
        for i in range(max_iterations):
            if stop_criteria(best.get_fitness()):
                print(
                    f"The genetic algorithm stops in the iteration = {i} with the stop criteria"
                )
                return best
            childrens = self.mate_population(population)  # Mating
            mutated_childrens = [children.mutate() for children in childrens]
            improved_childrens = [children.improve(5) for children in mutated_childrens]
            population = improved_childrens
            best = self.get_best(population)
        print(
            f"The algorithm stops after all the {max_iterations} and could not achieve the stop criteria"
        )
        return best

## Un Problema

Probemos este algoritmo con el problema de adivinar una palabra



In [296]:
available_letters = " abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ=?!-*+."
target_word = "Adivina esta palabra!!... Could this algorithm do it?"

letter_gene = ChoiceGene(available_values=available_letters)


def get_solution(character_list: list[str]) -> str:
    return "".join(character_list)


def get_word_fitness(character_list: list[str], target_word: str) -> float:
    attempt = get_solution(character_list)
    return sum(
        [1 for actual, expected in zip(attempt, target_word) if actual == expected]
    )


def generate_chromosome(
    letter_gene: ChoiceGene,
    target_word: str,
) -> SimpleChromosome:
    genes = [letter_gene.mutate() for _ in range(len(target_word))]
    fitness = lambda l: get_word_fitness(l, target_word)
    sample_chromosome = SimpleChromosome(
        genes, fitness_function=fitness, maximization_problem=True
    )
    return sample_chromosome

In [297]:
sample_chromosome = generate_chromosome(letter_gene, target_word)

genetic_algorithm = TournamentGeneticAlgorithm(
    sample_chromosome=sample_chromosome, tournament_percentage=0.3
)

solution = genetic_algorithm.solve(
    100, 500, stop_criteria=lambda fitness: fitness >= len(target_word)
)

real_solution = get_solution(solution.get_solution_values())
print(real_solution)

The genetic algorithm stops in the iteration = 53 with the stop criteria
Adivina esta palabra!!... Could this algorithm do it?


In [343]:
def fitness_2(l:list):
    company_name=l[0]
    val=l[1]
    new_=l[2]
    if company_name=="Primera" and val==4 and new_==9:
        return 20
    return 1.0
    

In [344]:
def generate_chromosome_2(
    gene_list: list[ChoiceGene],
    
) -> SimpleChromosome:
    genes = [v for v in gene_list]
    fitness = lambda l: gen_2
    sample_chromosome = SimpleChromosome(
        genes, fitness_function=fitness_2, maximization_problem=True
    )
    return sample_chromosome

In [345]:
class DeliveryInforme:
    pass

In [346]:
class To_Store:
    def __init__(self,name_type,name_companys:list[str]):
        self.name_type=name_type
        self.name_companys:list[str]=[]
        

In [347]:
class Organizer:
    def __init__(self):
        pass
        
    

In [352]:
company_names=["Segunda","Primera",]+[f'Value_{i}'for i in range(90)]

gen_1=ChoiceGene(company_names)
vals=[i for i in range(40)]
gen_2=ChoiceGene(vals)

gen_3=ChoiceGene(["kak"])

gen_4=ChoiceGene([i for i in range(90)])




In [353]:
def get_solution_2(lis:list):
    print(lis)

In [354]:
sample_chromosome = generate_chromosome_2([gen_1,gen_2,gen_3,gen_4])

genetic_algorithm = TournamentGeneticAlgorithm(
    sample_chromosome=sample_chromosome, tournament_percentage=0.3
)

solution = genetic_algorithm.solve(
    100, 500, stop_criteria=lambda fitness:fitness >= 20
)

real_solution = get_solution_2(solution.get_solution_values())
print(real_solution)

ValueError: Sample larger than population or is negative

## Como se puede usar esta API en nuestro proyecto?

Para nuestro problema de encontrar la mejor planificación a la hora de crear un producto y enviarlo a una tienda

Pensemos en un caso pequeño: Tenemos que producir un producto llamado Pizza con algún manufactor y para esto debemos enviarle la materia prima. Asumamos que la materia prima está completamente llena de productos bases. Digamos que los productos bases son:
- Tomate
- Masa de Pizza (asume que esto es un producto base)
- Queso

Y digamos que la los productores (empresas) de cada producto están en las siguientes listas:
- Tomate Producers
- Masa de Pizza Producers
- Queso Producers

La idea sería:
- crear un `ChoiceGene` por cada producto base:

```python
tomate_producers = [...] # Aqui esta la lista de empresas proveedoras que crean tomate
masa_producers = [...] # Aqui esta la lista de empresas proveedoras que crean masa de pizza
queso_producers = [...] # Aqui esta la lista de empresas proveedoras que crean queso

tomate_producer_gene = ChoiceGene(available_values=pizza_producers)
masa_producer_gene = ChoiceGene(available_values=masa_producers)
queso_producers_gene = ChoiceGene(available_values=queso_producers)
```

- crear un `ChoiceGene` que contenga la lista de empresas manufactoras de pizza

```python
pizza_manufactors = [...] # Aqui esta la lista de empresas manufactoras de pizza

pizza_manufactor_gene = ChoiceGene(available_values=pizza_manufactors)
```

- crear un `ChoiceGene` para los transportistas

```python
transportistas = [...] # Aqui va la lista de los transportistas

transportista_gene = ChoiceGene(available_values=transportistas)
```

- crear un cromosoma que contenga estos genes. Recuerda que el cromosoma representa una solución. Luego tu tienes que interpretar la solución del cromosoma. Ademas de que debes crear una función fitness que reciba una lista con los valores de los genes (en este caso son compañías o agentes, depende de ti).

```python

company_genes = [tomate_producer_gene, masa_producer_gene, queso_producers_gene, pizza_manufactor_gene, transportista_gene]

# Aqui debajo pon un lambda que recibe una lista de valores (compañías o agentes, depende de lo que hayas guardado en los genes) asignados de cada gen y devuelve un flotante que representa su fitness. Recuerda que es posible que para saber si una selección de empresas es buena tengas que preguntar los precios a cada empresa y la cantidad que venden, el precio del transportista, etc. Nuevamente, recuerda que todos los ingredientes de los proveedores deben ir a un manufactor (el dado en el gen de manufactores) por un agente transportista (el dado por el gen de transportista) y de este manufactor a la tienda que hizo la petición. De donde el fitness seria el costo de envió por estas empresas.
planification_fitness_function = ... 

# Como este problema busca el plan de menor costo es necesario asignar en False el parámetro `maximization_problem`
solution_chromosome = SimpleChromosome(genes, fitness_function=planification_fitness_function, maximization_problem=False)

```

- Crear una instancia del algoritmo genético `TournamentGeneticAlgorithm` para resolver el problema y obtener la solución

```python

# Tournament percentage representa el porcentaje de la población que se seleccionara random para simular torneos y asi generar
# los padres que se procrearan. Es bueno dejar una población de mas de 100 y un porcentaje de torneo de 0.3 (debe ser entre 0 y 1)
genetic_algorithm = TournamentGeneticAlgorithm(
    sample_chromosome=solution_chromosome, tournament_percentage=0.3
)

# Fíjate en el lambda que se usa para el `Stop criteria`. Esto es para parar el algoritmo genético si el fitness del mejor cromosoma
# cumple cierta condición. Aqui puse 500 como el máximo numero de iteraciones
solution = genetic_algorithm.solve(
    100, 500, stop_criteria=lambda fitness: fitness >= len(target_word)
)

# NOTA: La función get_solution se supone que la implementes como un método que convierte la lista de los valores de cada gen
# del cromosoma en una solución con la forma que desees. NO ES NECESARIO
real_solution = get_solution(solution.get_solution_values())
print(real_solution)
```